In [1]:
import cPickle
import time
import math
import random
import linecache
import numpy as np
import tensorflow as tf

from collections import Counter
from six.moves import xrange
from sklearn.metrics.pairwise import cosine_similarity as cos

In [2]:
qmr_path = '/home/jacobsuwang/Documents/UTA2017/BAYES-DL/WORD2VEC/DATA/qmr_freq.p'
    # concepts, cpt2prop, cpt2prop_prob, props
wiki_path = '/home/jacobsuwang/Documents/UTA2017/BAYES-DL/WORD2VEC/DATA/wiki2015_quadruples.txt'
    # doc_id, cpt, dep={nsubj,dobj,amod}, v
    # #lines = 267815247
wiki_concepts_path = '/home/jacobsuwang/Documents/UTA2017/BAYES-DL/WORD2VEC/DATA/wiki2015_qmrfreq.txt'
data_path = '/home/jacobsuwang/Documents/UTA2017/BAYES-DL/WORD2VEC/DATA/wiki2015_qmrfreq_data.txt'

In [10]:
# %%time

# # write wiki
# # include only the NPs in concepts
# # effective #lines = 8880645

concepts, cpt2prop, cpt2prop_prob, props = cPickle.load(open(qmr_path, 'r'))
concepts_set = set(concepts)

# count = 0
# start_time = time.time()

# with open(wiki_path, 'r') as source, open(wiki_concepts_path, 'w') as target:
#     for line in source:
#         line = line.split()
#         if len(line)!=4: continue
#         doc_id, cpt, dep, v = line
#         if cpt not in concepts_set: continue
#         vr = v+'-'+dep
#         target.write(cpt+' '+vr+'\n')
#         count += 1
#         if count%100000==0:
#             end_time = time.time()
#             print '... processed', count, 'lines', '('+str(end_time-start_time)+')'
#             start_time = end_time

In [8]:
# frequency filter

vr2count = Counter()

with open(wiki_concepts_path, 'r') as f:
    for line in f:
        line = line.split()
        if len(line)!=2: continue
        vr2count[line[1]] += 1
        
top50k_vrs = set([vr for vr,count in vr2count.most_common(50000)])

In [11]:
%%time

# encode targets (vr) and contexts (props)

vr2i = {}
prop2i = {}
def get_id(item, dic):
    if item not in dic:
        dic[item] = len(dic)
    return dic[item]

def sample_prop(cpt):
    props = cpt2prop[cpt]
    probs = np.array(cpt2prop_prob[cpt])
    unifs = np.random.uniform(0.,1.,len(probs))
    samps = np.sign(probs - unifs)
    return [props[i] for i,samp in enumerate(samps) if samp>0]

# simultaneous sampling example

# t = cpt2prop['cat']; print t
# a = np.array(cpt2prop_prob['cat']); print a
# b = np.random.uniform(0.,1.,len(cpt2prop_prob['cat'])); print b
# c = np.sign(a-b); print c
# s = [t[i] for i,c_i in enumerate(c) if c_i>0]
# print s

# [u'a_mammal', u'an_animal', u'a_pet', u'eats', u'has_4_legs', u'has_a_tail', u'has_claws', u'has_eyes', u'has_fur', u'has_legs', u'has_whiskers']
# [ 1.    1.    0.35  1.    1.    1.    1.    1.    1.    1.    1.  ]
# [ 0.38491228  0.72313118  0.37790994  0.14507703  0.60174872  0.29081557
#   0.0204469   0.60928764  0.91040059  0.97512716  0.58349327]
# [ 1.  1. -1.  1.  1.  1.  1.  1.  1.  1.  1.]
# [u'a_mammal', u'an_animal', u'eats', u'has_4_legs', u'has_a_tail', u'has_claws', u'has_eyes', u'has_fur', u'has_legs', u'has_whiskers']

count = 0
error_count = 0
entry_count = 0
start_time = time.time()

with open(wiki_concepts_path, 'r') as source, open(data_path, 'w') as target:
    for line in source:
        try:
            line = line.split()
            if len(line)!=2: continue
            cpt, vr = line
            if vr not in top50k_vrs: continue
            vr_id = get_id(vr, vr2i)
            props = sample_prop(cpt)
            for prop in props:
                prop_id = get_id(prop, prop2i)
                target.write(str(vr_id)+' '+str(prop_id)+'\n')
                entry_count += 1
        except:
            error_count += 1
        count += 1
        if count%100000==0:
            end_time = time.time()
            print '... processed', count, 'lines', '('+str(end_time-start_time)+')'
            start_time = end_time     
print 'effective #lines:', count
print 'effective #entries:', entry_count
print 'error #lines:', error_count
print 'vr dic size:', len(vr2i)
print 'prop dic size:', len(prop2i)

... processed 100000 lines (0.876940011978)
... processed 200000 lines (0.875649929047)
... processed 300000 lines (0.890516996384)
... processed 400000 lines (0.880336046219)
... processed 500000 lines (0.881725072861)
... processed 600000 lines (0.881027936935)
... processed 700000 lines (0.868283987045)
... processed 800000 lines (0.877635002136)
... processed 900000 lines (0.880290985107)
... processed 1000000 lines (0.867314100266)
... processed 1100000 lines (0.875893831253)
... processed 1200000 lines (0.887602090836)
... processed 1300000 lines (0.874397039413)
... processed 1400000 lines (0.874878883362)
... processed 1500000 lines (0.87594294548)
... processed 1600000 lines (0.873023033142)
... processed 1700000 lines (0.879861116409)
... processed 1800000 lines (0.880363941193)
... processed 1900000 lines (0.876384019852)
... processed 2000000 lines (0.885000944138)
... processed 2100000 lines (0.896829128265)
... processed 2200000 lines (0.869409799576)
... processed 230000

In [13]:
linecache.clearcache()

data_ids = range(entry_count) # also: wc -l datafile.txt
random.shuffle(data_ids)

class DataIterator:
    
    def __init__(self, data_ids):
        self.data_ids = data_ids
        self.epoch = 0
        self.cursor = 0
        self.size = len(data_ids)
        
    def get_batch(self, batch_ids):
        batch, labels = [], []
        for line_id in batch_ids:
            line = linecache.getline(data_path,line_id).split()
            if len(line)!=2: continue
            cpt_id,prop_id = line
            batch.append(int(cpt_id))
            labels.append([int(prop_id)])
        return np.array(batch),np.array(labels)
            
    def next_batch(self, n):
        if self.cursor+n > self.size:
            self.epoch += 1
            self.cursor = 0
            random.shuffle(self.data_ids)
        batch_ids = self.data_ids[self.cursor:self.cursor+n]
        self.cursor += n
        return self.get_batch(batch_ids)    

In [14]:
t = DataIterator(data_ids) # the first run is slow, but gets better.
tt = t.next_batch(10)
tt

(array([   75, 43068,   683,   457,  3820,   828,  6753,  2428,     4,   982]),
 array([[ 70],
        [ 38],
        [ 62],
        [174],
        [ 67],
        [ 93],
        [ 44],
        [109],
        [ 88],
        [ 88]]))

In [3]:
def reset_graph():
    if 'sess' in globals() and sess: sess.close()
    tf.reset_default_graph()
    
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15924146813224958251, name: "/gpu:0"
 device_type: "GPU"
 memory_limit: 10520248320
 locality {
   bus_id: 1
 }
 incarnation: 2722566146450705159
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0"]

In [105]:
reset_graph()

target_size = len(vr2i)
context_size = len(prop2i)
i2vr = {i:vr for vr,i in vr2i.iteritems()}
i2prop = {i:prop for prop,i in prop2i.iteritems()}

batch_size = 128
embedding_size = 128

test_vrs = ['kill-dobj','wear-dobj','eat-dobj','undo-dobj']

num_sampled = 64

graph = tf.Graph()

with graph.as_default():
    
    # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[None])
    train_labels = tf.placeholder(tf.int32, shape=[None, 1])
    
    # Test input
    test_input = tf.placeholder(tf.int32, shape=[1])
    
    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/gpu:0'):
        # Look up embeddings for inputs.
        embeddings = tf.Variable(
            tf.random_uniform([target_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

        # Construct the variables for the NCE loss
        nce_weights = tf.Variable(
            tf.truncated_normal([context_size, embedding_size],
                                stddev=1.0 / math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([context_size]))

    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=train_labels,
                       inputs=embed,
                       num_sampled=num_sampled,
                       num_classes=context_size))
    
    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.AdamOptimizer().minimize(loss)  
    
    ########################################################################
#     # Compute the cosine similarity between minibatch examples and all embeddings.
#     norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
#     normalized_embeddings = embeddings / norm
#     valid_embeddings = tf.nn.embedding_lookup(
#         normalized_embeddings, valid_dataset)
#     similarity = tf.matmul(
#         valid_embeddings, normalized_embeddings, transpose_b=True)
    ########################################################################
    
    # Prediction
    pred_emb = tf.reshape(tf.nn.embedding_lookup(embeddings, test_input), shape=[embedding_size])
    
    # Add variable initializer.
    init = tf.global_variables_initializer()

num_epochs = 1
num_steps = (int)((len(data_ids)/batch_size)*num_epochs)

with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    init.run()
    print('Initialized')
    
    gen = DataIterator(data_ids)

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = gen.next_batch(batch_size)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 1000 == 0:
            if step > 0:
                average_loss /= 1000
            # The average loss is an estimate of the loss over the last 1000 batches.
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0
        
        if step % 5000 == 0:
            try: assert exp in vr2i
            except: print 'UNKNOWN VR!' 
            current_embeddings = nce_weights.eval()
            for vr in test_vrs:
                inpt = np.array([vr2i[vr]])
                pred = session.run(pred_emb, feed_dict={test_input:inpt})
                pred = np.reshape(pred, [1,-1])
                vr_sim = cos(pred, current_embeddings)
                print vr
                print sorted([(i2prop[i],val) for i,val in enumerate(vr_sim[0])],
                             key=lambda x:x[1],reverse=True)[:5]

    final_embeddings = nce_weights.eval()

Initialized
('Average loss at step ', 0, ': ', 94.38299560546875)
kill-dobj
[(u'is_comfortable', 0.24348351), (u'used_for_protection', 0.22292781), (u'has_4_wheels', 0.21837823), (u'is_breakable', 0.1999481), (u'lives_in_trees', 0.18310645)]
wear-dobj
[(u'found_on_walls', 0.2190896), (u'floats', 0.20765409), (u'has_long_legs', 0.19328213), (u'used_for_holding_things', 0.19246447), (u'lives_in_water', 0.18009667)]
eat-dobj
[(u'sings', 0.30425447), (u'is_tall', 0.26243389), (u'is_shiny', 0.24277063), (u'found_in_kitchens', 0.21933906), (u'grows_in_gardens', 0.2171827)]
undo-dobj
[(u'is_grey', 0.24074695), (u'is_ugly', 0.21231258), (u'has_skin', 0.20050971), (u'worn_for_warmth', 0.1821624), (u'used_by_riding', 0.16784529)]
('Average loss at step ', 1000, ': ', 39.669471592903136)
('Average loss at step ', 2000, ': ', 14.124214435577393)
('Average loss at step ', 3000, ': ', 10.055346326351167)
('Average loss at step ', 4000, ': ', 8.2017953467369082)
('Average loss at step ', 5000, ': ', 

('Average loss at step ', 41000, ': ', 4.0026321327686309)
('Average loss at step ', 42000, ': ', 4.0052394561767581)
('Average loss at step ', 43000, ': ', 3.9949346003532411)
('Average loss at step ', 44000, ': ', 3.9846281714439393)
('Average loss at step ', 45000, ': ', 3.9944478299617767)
kill-dobj
[(u'a_carnivore', 0.048486587), (u'eats', 0.025014779), (u'an_animal', 0.024576085), (u'has_fur', 0.0076875854), (u'lives_in_Africa', -0.0016031906)]
wear-dobj
[(u'clothing', 0.14075814), (u'made_of_material', 0.090885431), (u'different_colours', 0.07516101), (u'worn_by_women', 0.065825008), (u'has_sleeves', 0.059803121)]
eat-dobj
[(u'is_edible', 0.099260807), (u'tastes_good', 0.072400682), (u'a_fruit', 0.054339424), (u'a_vegetable', 0.04745334), (u'has_skin', 0.039340992)]
undo-dobj
[(u'clothing', 0.057353981), (u'worn_by_men', 0.015211146), (u'made_of_silk', 0.003875182), (u'made_of_material', -0.0050108274), (u'is_comfortable', -0.012188051)]
('Average loss at step ', 46000, ': ', 3.

('Average loss at step ', 86000, ': ', 3.9274604475498198)
('Average loss at step ', 87000, ': ', 3.9246171157360079)
('Average loss at step ', 88000, ': ', 3.9341500966548919)
('Average loss at step ', 89000, ': ', 3.9322043616771696)
('Average loss at step ', 90000, ': ', 3.9308510637283325)
kill-dobj
[(u'a_carnivore', 0.054679148), (u'eats', 0.054553319), (u'an_animal', 0.032237161), (u'lives_in_Africa', 0.021119129), (u'lives_on_farms', 0.011404201)]
wear-dobj
[(u'clothing', 0.13132663), (u'has_sleeves', 0.080965564), (u'made_of_material', 0.080496006), (u'different_colours', 0.069103569), (u'worn_by_men', 0.063030012)]
eat-dobj
[(u'is_edible', 0.16960068), (u'is_white', 0.098604955), (u'eaten_by_baking', 0.084623612), (u'a_vegetable', 0.078518026), (u'is_nutritious', 0.06533961)]
undo-dobj
[(u'clothing', 0.19319241), (u'made_of_material', 0.093565732), (u'worn_by_men', 0.085222363), (u'is_long', 0.061782561), (u'made_of_silk', 0.039895713)]
('Average loss at step ', 91000, ': ', 3

('Average loss at step ', 131000, ': ', 3.9075956122875213)
('Average loss at step ', 132000, ': ', 3.9030434689521791)
('Average loss at step ', 133000, ': ', 3.9173924996852874)
('Average loss at step ', 134000, ': ', 3.9095532264709472)
('Average loss at step ', 135000, ': ', 3.8994192647933961)
kill-dobj
[(u'an_animal', 0.1108251), (u'has_4_legs', 0.065363169), (u'has_legs', 0.041434746), (u'has_fur', 0.025365975), (u'has_hair', 0.0042684753)]
wear-dobj
[(u'clothing', 0.13199709), (u'different_colours', 0.09280441), (u'made_of_material', 0.089218535), (u'worn_by_women', 0.070385747), (u'has_sleeves', 0.057876594)]
eat-dobj
[(u'is_edible', 0.086471111), (u'tastes_good', 0.077594087), (u'is_nutritious', 0.060486346), (u'a_vegetable', 0.052431025), (u'has_skin', 0.050464019)]
undo-dobj
[(u'clothing', 0.14760134), (u'made_of_different_materials', 0.06602525), (u'made_of_material', 0.044287223), (u'is_long', 0.031053498), (u'has_sleeves', 0.029147957)]
('Average loss at step ', 136000, 

('Average loss at step ', 176000, ': ', 3.9029541885852814)
('Average loss at step ', 177000, ': ', 3.8939751148223878)
('Average loss at step ', 178000, ': ', 3.8932503886222838)
('Average loss at step ', 179000, ': ', 3.8744028294086457)
('Average loss at step ', 180000, ': ', 3.9033950922489167)
kill-dobj
[(u'an_animal', 0.10869695), (u'has_4_legs', 0.061144404), (u'has_legs', 0.038549807), (u'a_predator', 0.037811913), (u'eats_grass', 0.035340369)]
wear-dobj
[(u'clothing', 0.11071023), (u'made_of_material', 0.10237838), (u'has_sleeves', 0.089118384), (u'different_colours', 0.087188244), (u'worn_by_women', 0.072905019)]
eat-dobj
[(u'is_edible', 0.10671558), (u'tastes_good', 0.077281736), (u'is_yellow', 0.072576888), (u'is_nutritious', 0.06801232), (u'cannot_fly', 0.054198794)]
undo-dobj
[(u'is_long', 0.076890416), (u'clothing', 0.070369013), (u'has_sleeves', 0.051286925), (u'made_of_different_materials', 0.043925118), (u'is_smooth', 0.043071415)]
('Average loss at step ', 181000, ':

('Average loss at step ', 221000, ': ', 3.8901906807422639)
('Average loss at step ', 222000, ': ', 3.893807150363922)
('Average loss at step ', 223000, ': ', 3.8877615263462069)
('Average loss at step ', 224000, ': ', 3.8892508623600004)
('Average loss at step ', 225000, ': ', 3.8838310012817381)
kill-dobj
[(u'an_animal', 0.094443403), (u'has_4_legs', 0.085003123), (u'eats', 0.068548493), (u'has_hair', 0.067625865), (u'a_predator', 0.03653669)]
wear-dobj
[(u'clothing', 0.13952905), (u'worn_by_women', 0.10971188), (u'different_colours', 0.097357519), (u'has_sleeves', 0.08460936), (u'made_of_material', 0.084279716)]
eat-dobj
[(u'is_edible', 0.099326186), (u'is_nutritious', 0.076100476), (u'tastes_good', 0.074891835), (u'is_white', 0.066816889), (u'eaten_by_baking', 0.063051358)]
undo-dobj
[(u'clothing', 0.10920686), (u'is_long', 0.072604425), (u'made_of_material', 0.040633809), (u'worn_by_men', 0.040162597), (u'different_colours', 0.024762049)]
('Average loss at step ', 226000, ': ', 3.

('Average loss at step ', 266000, ': ', 3.8801494271755219)
('Average loss at step ', 267000, ': ', 3.8977716836929321)
('Average loss at step ', 268000, ': ', 3.8938753161430357)
('Average loss at step ', 269000, ': ', 3.8902039201259613)
('Average loss at step ', 270000, ': ', 3.879091825723648)
kill-dobj
[(u'an_animal', 0.12176893), (u'has_4_legs', 0.077721886), (u'a_mammal', 0.050784081), (u'has_a_tail', 0.046224814), (u'has_fur', 0.045449678)]
wear-dobj
[(u'clothing', 0.1646896), (u'different_colours', 0.11896645), (u'worn_by_women', 0.11380026), (u'made_of_material', 0.10155357), (u'has_sleeves', 0.088531107)]
eat-dobj
[(u'tastes_good', 0.093080312), (u'is_edible', 0.07914447), (u'is_yellow', 0.074890673), (u'is_white', 0.056844167), (u'is_nutritious', 0.045969721)]
undo-dobj
[(u'clothing', 0.096014656), (u'is_smooth', 0.082393125), (u'has_sleeves', 0.070157632), (u'is_long', 0.068536989), (u'made_of_material', 0.052371375)]
('Average loss at step ', 271000, ': ', 3.8917930488586

('Average loss at step ', 311000, ': ', 3.876714286327362)
('Average loss at step ', 312000, ': ', 3.8781005840301512)
('Average loss at step ', 313000, ': ', 3.87982954287529)
('Average loss at step ', 314000, ': ', 3.8821381886005399)
('Average loss at step ', 315000, ': ', 3.8799875509738921)
kill-dobj
[(u'an_animal', 0.15541087), (u'has_4_legs', 0.10731964), (u'a_carnivore', 0.057994936), (u'a_mammal', 0.03907479), (u'has_a_tail', 0.036551401)]
wear-dobj
[(u'worn_by_women', 0.1151563), (u'clothing', 0.11495078), (u'different_colours', 0.11208773), (u'worn_by_men', 0.079486653), (u'made_of_material', 0.076480448)]
eat-dobj
[(u'is_white', 0.11916877), (u'is_edible', 0.11180584), (u'is_nutritious', 0.073372439), (u'has_skin', 0.064488918), (u'a_vegetable', 0.057676896)]
undo-dobj
[(u'eats_fish', 0.15266988), (u'clothing', 0.06752155), (u'worn_by_women', 0.040719155), (u'lives_in_water', 0.03414106), (u'made_of_material', 0.02492285)]
('Average loss at step ', 316000, ': ', 3.871436214

In [ ]:
# def get_top_props(exp):
    
#     try: assert exp in vr2i
#     except: print 'UNKNOWN VR!'
        
#     inpt = np.array([vr2i[exp]])
    
#     with tf.Session(graph=graph) as sess:
#         sess.run(tf.global_variables_initializer())
#         pred = sess.run(pred_emb, feed_dict={test_input:inpt})
#         pred = np.reshape(pred, [1, -1])
    
#     exp_sim = cos(pred, final_embeddings)
#     print sorted([(i2prop[i],val) for i,val in enumerate(exp_sim[0])],key=lambda x:x[1],reverse=True)[:5]